# Resolution of the optimisation problem

This notebook will help solve the following optimisation problem : 
TODO

In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print("PROJECT_ROOT =", PROJECT_ROOT)


PROJECT_ROOT = c:\Users\marie\OneDrive - edu.escpeurope.eu\CS\Filière\avencore_hydrogen_production


## 1. Imports initiaux

In [2]:
import pyomo.environ as pyo

from Data.loading import load_prices
from Battery.battery_simulation import BatteryParams, BatteryBlock
from Electrolyser.electrolyser_simulation import ElectrolyserParams, ElectrolyserBlock, PowerBalanceBlock
from Costs.cost_functions import EconomicParams, ObjectiveBuilder

## 2. Loading the dataset

In [3]:
path_price = "../Data/prix_spot.csv"
prix_elec, intensite_co2, T, df_2024 = load_prices(path_price, year=2024)

print(df_2024.head())

                      Date  Spot  T_weight  W_100
192840 2024-01-01 00:00:00  0.10       6.7   33.0
192841 2024-01-01 01:00:00  0.01       6.7   31.0
192842 2024-01-01 02:00:00  0.00       6.6   30.0
192843 2024-01-01 03:00:00 -0.01       6.4   30.0
192844 2024-01-01 04:00:00 -0.03       6.3   29.0


## 3. Global parameters

In [4]:
dt = 1.0
econ = EconomicParams(
    c_bat_P = 150.0,
    c_bat_E = 10_000.0,
    prix_a_terme = 75.0,
    c_CO2 = 80.0
)

el_params = ElectrolyserParams(
    P_hydro_max = 100.0,
    u_el_min = 0.10,
    u_el_max = 0.95,
    r_el = 1.0,
    eta_el = 0.70,
    LHV_H2 = 55.0,
    dt = dt,
    H2_target = 1_000_000.0
)

bat_params = BatteryParams(
    eta_ch = 1.0,
    eta_dis = 1.0,
    dt = dt,
    SOC_min = 0.10,
    SOC_max = 0.95
)

## 4. Pyomo model definition

In [5]:
m = pyo.ConcreteModel()
m.T = pyo.RangeSet(0, T-1)

# Dimension variables
m.P_bat_max = pyo.Var(domain=pyo.NonNegativeReals) # MW
m.E_bat_max = pyo.Var(domain=pyo.NonNegativeReals) # MWh

## 5. Adding the different blocs

In [6]:
# Battery
bat = BatteryBlock(m, m.T, bat_params)
bat.add_variables()
bat.add_constraints()

# PEM Electrolyser
el = ElectrolyserBlock(m, m.T, el_params)
el.add_variables()
el.add_constraints()

# Power balance
pb = PowerBalanceBlock(m, m.T, P_hydro_max=el_params.P_hydro_max, phi_bounds=(0.0,1.0))
pb.add_variables()
pb.add_constraints()

## 6. Objective definition and solving

In [7]:
objb = ObjectiveBuilder(m, m.T, econ, prix_elec=prix_elec, intensite_co2=intensite_co2, dt=dt, include_co2=False)
objb.build_objective()

solver = pyo.SolverFactory("highs")
res = solver.solve(m, tee=True)
print(res.solver.status, res.solver.termination_condition)

Running HiGHS 1.12.0 (git hash: 755a8e0): Copyright (c) 2025 HiGHS under MIT licence terms
LP has 34560 rows; 20739 cols; 82940 nonzeros
Coefficient ranges:
  Matrix  [1e-01, 1e+02]
  Cost    [1e-02, 3e+05]
  Bound   [1e+00, 1e+00]
  RHS     [1e+00, 1e+06]
Presolving model
24191 rows, 17282 cols, 69114 nonzeros  0s
Dependent equations search running on 4878 equations with time limit of 1000.00s
Dependent equations search removed 0 rows and 0 nonzeros in 0.00s (limit = 1000.00s)
22156 rows, 15249 cols, 74936 nonzeros  0s
Presolve reductions: rows 22156(-12404); columns 15249(-5490); nonzeros 74936(-8004) 
Solving the presolved LP
Using EKK dual simplex solver - serial
  Iteration        Objective     Infeasibilities num(sum)
          0    -6.0153876423e+01 Ph1: 160(1314); Du: 160(60.1539) 0s
      10389     4.7107457089e+04 Pr: 0(0) 3s

Performed postsolve
Solving the original LP from the solution after postsolve

Model status        : Optimal
Simplex   iterations: 10389
Objective valu

## 7. Results

In [8]:
P_hydro_moy = sum(pyo.value(m.P_hydro[t]) for t in m.T) / T
P_hydro_tot = sum(pyo.value(m.P_hydro[t]) for t in m.T)
H2_tot_opt  = sum(pyo.value(m.H2[t]) for t in m.T)
J_opt = pyo.value(m.Obj)
cout_moyen = (sum(econ.prix_a_terme * pyo.value(m.phi) * dt + prix_elec[t] * pyo.value(m.P_spot[t]) * dt for t in m.T) / max(P_hydro_tot, 1e-9))
LCOH_opt = J_opt / max(H2_tot_opt, 1e-9)

print("Electrolyser max power (MW):", f"{el_params.P_hydro_max:.2f}")
print("Optimal battery power (MW):", f"{pyo.value(m.P_bat_max):.2f}")
print("Optimal battery capacity (MWh):", f"{pyo.value(m.E_bat_max):.2f}")
print("Optimal percentage forward (%):", f"{pyo.value(m.phi)*100:.2f}")
print("Mean power electrolyser (MW):", f"{P_hydro_moy:.2f}")
print("Total H2 production (kg):", f"{H2_tot_opt:.2f}")
print("Total cost (€):", f"{J_opt:.2f}")
print("LCOH (€/kg):", f"{LCOH_opt:.2f}")

Electrolyser max power (MW): 100.00
Optimal battery power (MW): 1.00
Optimal battery capacity (MWh): 1.18
Optimal percentage forward (%): 22.07
Mean power electrolyser (MW): 22.73
Total H2 production (kg): 1000000.00
Total cost (€): 47107.46
LCOH (€/kg): 0.05
